In [1]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests
import warnings
warnings.filterwarnings('ignore')
import pymongo

# Web Scrapping

In [73]:
# URL of page to be scraped
url = 'https://www.bls.gov/oes/current/oessrcst.htm#a'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html'
soup = BeautifulSoup(response.text, 'html')

In [74]:
results = soup.find_all('blockquote')

In [75]:
list_links=[]
for result in results:
    link=result.a['href']
    final_link = f'https://www.bls.gov/oes/current/{link}'
    list_links.append(final_link)
    
list_links

['https://www.bls.gov/oes/current/oes_al.htm',
 'https://www.bls.gov/oes/current/oes_ca.htm',
 'https://www.bls.gov/oes/current/oes_de.htm',
 'https://www.bls.gov/oes/current/oes_fl.htm',
 'https://www.bls.gov/oes/current/oes_ga.htm',
 'https://www.bls.gov/oes/current/oes_hi.htm',
 'https://www.bls.gov/oes/current/oes_id.htm',
 'https://www.bls.gov/oes/current/oes_ks.htm',
 'https://www.bls.gov/oes/current/oes_la.htm',
 'https://www.bls.gov/oes/current/oes_me.htm',
 'https://www.bls.gov/oes/current/oes_ne.htm',
 'https://www.bls.gov/oes/current/oes_oh.htm',
 'https://www.bls.gov/oes/current/oes_pa.htm',
 'https://www.bls.gov/oes/current/oes_ri.htm',
 'https://www.bls.gov/oes/current/oes_sc.htm',
 'https://www.bls.gov/oes/current/oes_tn.htm',
 'https://www.bls.gov/oes/current/oes_ut.htm',
 'https://www.bls.gov/oes/current/oes_vt.htm',
 'https://www.bls.gov/oes/current/oes_wa.htm']

In [76]:
#selecting 3 states
ca_fl_pa_list = []
for item in list_links:
    if 'ca' in item or 'fl' in item or 'pa' in item:
        ca_fl_pa_list.append(item)
ca_fl_pa_list

['https://www.bls.gov/oes/current/oes_ca.htm',
 'https://www.bls.gov/oes/current/oes_fl.htm',
 'https://www.bls.gov/oes/current/oes_pa.htm']

In [77]:
data_df= [pd.read_html(state) for state in ca_fl_pa_list]

In [78]:
l=[]
for i in range(len(data_df)):
    l.append(data_df[i][0])

# Cleaning the data

In [79]:

cl_df=l[0]
fl_df=l[1]
pa_df=l[2]

In [80]:
cl_df['state'] =  'CA'
fl_df['state'] =  'FL' 
pa_df['state'] =  'PA' 


In [81]:
cl_fl=cl_df.append(fl_df)
cl_fl_pa = cl_fl.append(pa_df)

In [82]:
ocupations_cl_fl_pa =cl_fl_pa.loc[cl_fl_pa['Level']=='major',:]

In [83]:
ocupations_cl_fl_pa.reset_index(inplace=True,drop=True)

In [84]:
ocupations_cl_fl_pa.head()

,Occupation code,Occupation title (click on the occupation title to view its profile),Level,Employment,Employment RSE,"Employment per 1,000 jobs",Location quotient,Median hourly wage,Mean hourly wage,Annual mean wage,Mean wage RSE,state
0,11-0000,Management Occupations,major,996830,0.6%,58.610,1.11,$57.43,$65.31,"$135,840",0.4%,CA
1,13-0000,Business and Financial Operations Occupations,major,1021960,0.7%,60.088,1.13,$35.62,$40.05,"$83,310",0.4%,CA
2,15-0000,Computer and Mathematical Occupations,major,616270,1.2%,36.235,1.20,$49.67,$52.01,"$108,190",0.9%,CA
3,17-0000,Architecture and Engineering Occupations,major,337180,1.3%,19.825,1.12,$45.85,$48.46,"$100,800",0.7%,CA
4,19-0000,"Life, Physical, and Social Science Occupations",major,177450,2.5%,10.434,1.29,$38.18,$41.18,"$85,640",1.2%,CA


In [85]:
for col in ocupations_cl_fl_pa[['Employment','Employment RSE','Employment per 1,000 jobs','Median hourly wage','Mean wage RSE','Mean hourly wage','Annual mean wage']]:
    ocupations_cl_fl_pa[col]=ocupations_cl_fl_pa[col].apply(lambda x: x.replace('$','').replace('(','').replace('%','').replace(')','').replace(',','')).astype(float)


In [86]:
ocupations_cl_fl_pa['Occupation code'] = ocupations_cl_fl_pa['Occupation code'].apply(lambda x:x.replace('-','')).\
astype(int)

# Uploading to MongoDB

In [88]:
# create client
client=pymongo.MongoClient('mongodb://localhost:27017')
#create DB
db = client.beareau_etl
collection = db.ca_fl_pa

In [91]:
#convert df to dict
dict_to_upload = ocupations_cl_fl_pa.to_dict(orient='records')


In [92]:
db.ca_fl.pa.insert(dict_to_upload)

[ObjectId('5e1cfff3aa46595943b507d1'),
 ObjectId('5e1cfff3aa46595943b507d2'),
 ObjectId('5e1cfff3aa46595943b507d3'),
 ObjectId('5e1cfff3aa46595943b507d4'),
 ObjectId('5e1cfff3aa46595943b507d5'),
 ObjectId('5e1cfff3aa46595943b507d6'),
 ObjectId('5e1cfff3aa46595943b507d7'),
 ObjectId('5e1cfff3aa46595943b507d8'),
 ObjectId('5e1cfff3aa46595943b507d9'),
 ObjectId('5e1cfff3aa46595943b507da'),
 ObjectId('5e1cfff3aa46595943b507db'),
 ObjectId('5e1cfff3aa46595943b507dc'),
 ObjectId('5e1cfff3aa46595943b507dd'),
 ObjectId('5e1cfff3aa46595943b507de'),
 ObjectId('5e1cfff3aa46595943b507df'),
 ObjectId('5e1cfff3aa46595943b507e0'),
 ObjectId('5e1cfff3aa46595943b507e1'),
 ObjectId('5e1cfff3aa46595943b507e2'),
 ObjectId('5e1cfff3aa46595943b507e3'),
 ObjectId('5e1cfff3aa46595943b507e4'),
 ObjectId('5e1cfff3aa46595943b507e5'),
 ObjectId('5e1cfff3aa46595943b507e6'),
 ObjectId('5e1cfff3aa46595943b507e7'),
 ObjectId('5e1cfff3aa46595943b507e8'),
 ObjectId('5e1cfff3aa46595943b507e9'),
 ObjectId('5e1cfff3aa4659

In [97]:
listings=db.ca_fl.pa.find()

In [98]:
for listing in listings:
    print(listing)

{'_id': ObjectId('5e1cfff3aa46595943b507d1'), 'Occupation code': 110000, 'Occupation title (click on the occupation title to view its profile)': 'Management Occupations', 'Level': 'major', 'Employment': 996830.0, 'Employment RSE': 0.6, 'Employment per 1,000 jobs': 58.61, 'Location quotient': '1.11', 'Median hourly wage': 57.43, 'Mean hourly wage': 65.31, 'Annual mean wage': 135840.0, 'Mean wage RSE': 0.4, 'state': 'CA'}
{'_id': ObjectId('5e1cfff3aa46595943b507d2'), 'Occupation code': 130000, 'Occupation title (click on the occupation title to view its profile)': 'Business and Financial Operations Occupations', 'Level': 'major', 'Employment': 1021960.0, 'Employment RSE': 0.7, 'Employment per 1,000 jobs': 60.088, 'Location quotient': '1.13', 'Median hourly wage': 35.62, 'Mean hourly wage': 40.05, 'Annual mean wage': 83310.0, 'Mean wage RSE': 0.4, 'state': 'CA'}
{'_id': ObjectId('5e1cfff3aa46595943b507d3'), 'Occupation code': 150000, 'Occupation title (click on the occupation title to vie